# Exact Solution for Job Shop Problem

Данная модель находит решение JSP с помощью вспомогательной утилиты cp_model из пакета Google Operation Research, которая позволяет организовать перебор из области допустимых значений учитывая ограничивающие условия. Также с ее помощью можно найти такие допустимые значения при которых наперед заданая функция достигает минимального (максимального) значения. Именно этими фунциями библиотеки мы и будем пользоваться.

In [1]:
import collections
from ortools.sat.python import cp_model

### Условия задачи

In [2]:
jobs_data = [[(0, 7), (1, 5), (2, 2), (3, 3), (4, 9)],
       [(0, 6), (1, 6), (2, 4), (3, 5), (4, 10)],
       [(0, 5), (1, 4), (2, 5), (3, 6), (4, 8)],
       [(0, 8), (1, 3), (2, 3), (3, 2), (4, 6)]]

machines_count = 1 + max(task[0] for job in jobs_data for task in job)
all_machines = range(machines_count)

# Оценка сверху для области значения всех переменных (сумма всего времени выполнения)
horizon = sum(task[1] for job in jobs_data for task in job)

Обьявляем переменные start и end для каждой подзадачи, и переменную интервала, которая работает, как связь между start и end вида:

**start + duration = end**

In [3]:
model = cp_model.CpModel()
all_tasks = {}
machine_to_intervals = collections.defaultdict(list)

task_type = collections.namedtuple('task_type', 'start end interval')
assigned_task_type = collections.namedtuple('assigned_task_type', 'start job index duration')

for job_id, job in enumerate(jobs_data):
    for task_id, task in enumerate(job):
        machine = task[0]
        duration = task[1]
        suffix = '_%i_%i' % (job_id, task_id)
        start_var = model.NewIntVar(0, horizon, 'start' + suffix)
        end_var = model.NewIntVar(0, horizon, 'end' + suffix)
        interval_var = model.NewIntervalVar(start_var, duration, end_var, 'interval' + suffix)
        # Создаем лист, ключем которого будет кортеж (джоб_айди, таск,_айди),а значениям будет именованный кортеж
        # с обьектами переменных (начало, конец, интервал)
        all_tasks[job_id, task_id] = task_type(
            start=start_var, end=end_var, interval=interval_var)
        # Каждой машине ставим в соотвествие интервал таска
        machine_to_intervals[machine].append(interval_var)


*В нашей задаче есть следующие ограничения:*
- Для каждой подзадачи время начала подзадачи + ее длительность меньше или равно времении начала следующей подзадачи 
- Интервалы работы подзадач каждой машины не пересекааются 

In [4]:
# Ограничение на непересечение интервалов
for machine in all_machines:
    model.AddNoOverlap(machine_to_intervals[machine])

# Ограничение на конец n-ой и начало n+1-ой подзадачи в одной задаче
for job_id, job in enumerate(jobs_data):
    for task_id in range(len(job) - 1):
        model.Add(all_tasks[job_id, task_id + 1].start >= all_tasks[job_id, task_id].end)


Добавим целевую переменную, которую будем затем минимизировать. Это будет максимум концов подзадач.

In [5]:
# Переменная временных затрат
obj_var = model.NewIntVar(0, horizon, 'makespan')

# Добавляем условие obj_var == max(all_tasks.end)
model.AddMaxEquality(obj_var, [
    all_tasks[job_id, len(job) - 1].end
    for job_id, job in enumerate(jobs_data)
])
# Минимизируем временные затраты
model.Minimize(obj_var)

# Запускаем перебор
solver = cp_model.CpSolver()
status = solver.Solve(model)

### Вывод результатов

In [6]:
# Если нашли оптимальное решение
if status == cp_model.OPTIMAL:
    # Создаем для каждой машины лист тасков и оптимального решения
    assigned_jobs = collections.defaultdict(list)
    for job_id, job in enumerate(jobs_data):
        for task_id, task in enumerate(job):
            machine = task[0]
            assigned_jobs[machine].append(
                assigned_task_type(
                    start=solver.Value(all_tasks[job_id, task_id].start),
                    job=job_id,
                    index=task_id,
                    duration=task[1]))
    # Вывод для задачи
    output = ''
    for machine in all_machines:
        # Сортируем по времени
        assigned_jobs[machine].sort()
        sol_line_tasks = 'Машина ' + str(machine) + ': '
        sol_line = '          '
        
        for assigned_task in assigned_jobs[machine]:
            name = 'подзадача_%i_%i' % (assigned_task.job, assigned_task.index)
            sol_line_tasks += '%-20s' % name
            start = assigned_task.start
            duration = assigned_task.duration
            sol_tmp = '[%i,%i]' % (start, start + duration)
            sol_line += '%-20s' % sol_tmp
            
        sol_line += '\n'
        sol_line_tasks += '\n'
        output += sol_line_tasks
        output += sol_line

    print('Время оптимального решения: %i' % solver.ObjectiveValue())
    print('\n')
    print(output)

Время оптимального решения: 51


Машина 0: подзадача_0_0       подзадача_2_0       подзадача_1_0       подзадача_3_0       
          [0,7]               [7,12]              [12,18]             [18,26]             
Машина 1: подзадача_0_1       подзадача_2_1       подзадача_1_1       подзадача_3_1       
          [7,12]              [12,16]             [18,24]             [26,29]             
Машина 2: подзадача_0_2       подзадача_2_2       подзадача_1_2       подзадача_3_2       
          [12,14]             [16,21]             [24,28]             [29,32]             
Машина 3: подзадача_0_3       подзадача_2_3       подзадача_1_3       подзадача_3_3       
          [14,17]             [21,27]             [28,33]             [33,35]             
Машина 4: подзадача_0_4       подзадача_2_4       подзадача_1_4       подзадача_3_4       
          [17,26]             [27,35]             [35,45]             [45,51]             

